In [1]:
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras.preprocessing import image
import numpy as np
import tensorflow as tf
from tensorflow.python import keras
import numpy as np
import sys
from PIL import Image
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

tf.__version__

'1.8.0'

In [7]:
data_path = ["ddsm-mammography/training10_0.tfrecords",
             "ddsm-mammography/training10_1.tfrecords",
             "ddsm-mammography/training10_2.tfrecords",
             "ddsm-mammography/training10_3.tfrecords",
             "ddsm-mammography/training10_4.tfrecords"]

with tf.Session() as sess:
    features={
            'label': tf.FixedLenFeature([], tf.int64),
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        }
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer(data_path, num_epochs=1)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=features)
    
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image'], tf.uint8)
    
    # Cast label data into int32
    label = features['label_normal']
    
    # Reshape image data into the original shape
    image = tf.reshape(image, [299, 299, 1])
    
    # Any preprocessing here ...
    
    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=5, capacity=30, num_threads=1, min_after_dequeue=10)
        
    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    #make directories to store files
    pos_dir = os.path.join(os.getcwd(), "mammo\\train\\pos")
    pos_test_dir = os.path.join(os.getcwd(), "mammo\\test\\pos")
    pos_valid_dir = os.path.join(os.getcwd(), "mammo\\validation\\pos")
    neg_dir = os.path.join(os.getcwd(), "mammo\\train\\neg")
    neg_test_dir = os.path.join(os.getcwd(), "mammo\\test\\neg")
    neg_valid_dir = os.path.join(os.getcwd(), "mammo\\validation\\neg")
     
    directories = [pos_dir, pos_test_dir, pos_valid_dir, neg_dir, neg_test_dir, neg_valid_dir]
    for x in directories:
        os.makedirs(x, exist_ok=True)
        
    #percent to split data
    pct = .15
    
    test_neg = int(pct * 48956)
    test_pos = int(pct * 7289)
    
    test_neg2 = int((pct + pct) * 48956)
    test_pos2 = int((pct + pct) * 7289)
    
    n = 0
    p = 0
    for batch_index in range(11177):
        img, lbl = sess.run([images, labels])
        #img = img.astype(np.uint8)
        for j in range(5):
            
            if lbl[j] == 1 and p <= test_pos:
                p += 1
                pillow_image = Image.fromarray(img[j].reshape([299, 299]))
                pillow_image.save(('mammo\\test\\pos\\positive' + str(p) + '.jpg'))
            elif lbl[j] == 1 and p <= test_pos2:
                p += 1
                pillow_image = Image.fromarray(img[j].reshape([299, 299]))
                pillow_image.save(('mammo\\validation\\pos\\positive' + str(p) + '.jpg'))
            elif lbl[j] == 1:
                p += 1
                pillow_image = Image.fromarray(img[j].reshape([299, 299]))
                pillow_image.save(('mammo\\train\\pos\\positive' + str(p) + '.jpg'))
            elif lbl[j] == 0 and n <= test_neg:
                n += 1
                pillow_image = Image.fromarray(img[j].reshape([299, 299]))
                pillow_image.save(('mammo\\test\\neg\\negative' + str(n) + '.jpg'))
            elif lbl[j] == 0 and n <= test_neg2:
                n += 1
                pillow_image = Image.fromarray(img[j].reshape([299, 299]))
                pillow_image.save(('mammo\\validation\\neg\\negative' + str(n) + '.jpg'))
            elif lbl[j] == 0:
                n += 1
                pillow_image = Image.fromarray(img[j].reshape([299, 299]))
                pillow_image.save(('mammo\\train\\neg\\negative' + str(n) + '.jpg'))
            #plt.subplot(2, 3, j+1)
            #plt.title(lbl[j])
        #plt.show()
        
    # Stop the threads
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)
    sess.close()